In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm_notebook
from sklearn import preprocessing 
from sklearn.impute import KNNImputer
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
import random
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBRegressor
import lightgbm as lgb

In [2]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kagglesdsdata/competitions/6392/44054/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1601614655&Signature=kdekCWtvrhiCUc3vLx%2Bu9CIHOP2uIvWVw%2FgVNcuHkjYRCsgKNQZmdPFSozQVeYkXkyrA83vk3JhgX5GilsU580rXcfDcgEkp5W%2BAAuouX5KSiHcGsSAr7pHpwQ0Qn1wNCnI4%2F9vrp7k7JpRMbh0PG%2Fj8miXWxfx6CMba0kgBXGjQGTSmU1SfMdzwyHJMrf7rpYOR8R5om5ycBcOsnG9k%2FfnHYCqE4CTanO%2BndjiHeGUYI3pAkdeB5Kw2kLq51cZd8N9NgZhXugcPkid1dmZ8ZTTUqsiZhRocc3NrwI90RABuEJmdcwyXchAtue%2FImDeb0kSxXwD7cSn70GGmP569yQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip" -c -O 'train.csv.zip'
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kagglesdsdata/competitions/6392/44054/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1601614691&Signature=DcNgCcbI9E0whdF4TwcffSDmaI3s1kTNU2dJy9qq3aVsxskb%2FjWXtMqd84pZZBU87vgm9cFTFtrfmSV8Zme3teJVEV9IF4dKs6EUBQklPqo37ZcTclO%2B6EI7oW9w5bAxuOF1Mnu3%2FDc%2Bd1XJBjADPYJun8uyAHhFXSuk7qC8ESk5rHEcw4c9VnVN2dWNAz8Nu6VMkvzvfFJVlRIVMAVPwzNErF5BkR2Pc%2BtTSw4cHNEsG5GiMevYZ9C%2FXq3YePn0lXDMlYfuJDLGoRg%2BOaapxqJWm1bGQ8L8Pkb3ZPY5Bi1kvL9NWS75fqMb7CZ4zsBZ38sTtTzV06xBf%2BPyG9mDBg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtest.csv.zip" -c -O 'test.csv.zip'

--2020-09-30 17:48:58--  https://storage.googleapis.com/kagglesdsdata/competitions/6392/44054/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1601614655&Signature=kdekCWtvrhiCUc3vLx%2Bu9CIHOP2uIvWVw%2FgVNcuHkjYRCsgKNQZmdPFSozQVeYkXkyrA83vk3JhgX5GilsU580rXcfDcgEkp5W%2BAAuouX5KSiHcGsSAr7pHpwQ0Qn1wNCnI4%2F9vrp7k7JpRMbh0PG%2Fj8miXWxfx6CMba0kgBXGjQGTSmU1SfMdzwyHJMrf7rpYOR8R5om5ycBcOsnG9k%2FfnHYCqE4CTanO%2BndjiHeGUYI3pAkdeB5Kw2kLq51cZd8N9NgZhXugcPkid1dmZ8ZTTUqsiZhRocc3NrwI90RABuEJmdcwyXchAtue%2FImDeb0kSxXwD7cSn70GGmP569yQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 142.250.107.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17919166 (17M) [application/zip]
Saving to: ‘train.csv.zip’

train.csv.zip       100%[=================

In [3]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
   creating: __MACOSX/
  inflating: __MACOSX/._train.csv    
Archive:  test.csv.zip
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     


In [4]:
def final_fun_1(X):

  # Drop features that has little to no variance :

  zero_variance = ["culture_objects_top_25_raion", "oil_chemistry_raion", "railroad_terminal_raion", "nuclear_reactor_raion",
      "build_count_foam", "big_road1_1line", "railroad_1line", "office_sqm_500", "trc_sqm_500",
      "cafe_count_500_price_4000", "cafe_count_500_price_high", "mosque_count_500", "leisure_count_500",
      "office_sqm_1000", "trc_sqm_1000", "cafe_count_1000_price_high", "mosque_count_1000", "cafe_count_1500_price_high",
      "mosque_count_1500", "cafe_count_2000_price_high"]
  
  useless_ids = ["ID_metro", "ID_railroad_station_walk", "ID_railroad_station_avto", "ID_big_road1", "ID_big_road2",
      "ID_railroad_terminal", "ID_bus_terminal"]

  X.drop(zero_variance + useless_ids, axis = 1, inplace = True)

  # Replace Irrevant data from relevant one

  # State
  X.state.replace({33:3},inplace=True)

  # Material
  X['material'].replace(to_replace = 3, value = 1, inplace = True)

  # full_sq
  X['full_sq'].replace(to_replace = 0, value = np.nan, inplace = True)

  # Max Floor
  X['max_floor'].replace(to_replace = 0, value = np.nan, inplace = True)

  # Number of Rooms
  X['num_room'].replace(to_replace = 0, value = np.nan, inplace = True)

  # Feature Engineering #

  #print("Doing Feature Engineering ")

  # Additional timestamp variables:
  X['year'] = X['timestamp'].apply(lambda x: int(x[0:4]))
  X['year_mo'] = X['timestamp'].apply(lambda x: x[0:7])


  # Residential & kitchen area to total area ratio:
  X['resident_to_total_ratio'] = X['life_sq']/X['full_sq']

  # Average area per room:
  X['avg_room_area'] = X['life_sq']/X['num_room']

  # Extra area:
  X['extra_area'] = X['full_sq'] - X['life_sq']
  X['extra_area_ratio'] = X['extra_area']/X['full_sq']

  # Percentage of population in labor force:
  X['pct_labor_force'] = X['work_all']/X['raion_popul']

  # Apartment floor relative to building height:
  X['floor_rel_total'] = X['floor']/X['max_floor']


  # Some additional binary variables:
  X['metro_flag'] = np.where(X['raion_popul'] > 150000, 1, 0)
  X['large_flag'] = np.where(X['max_floor'] >= 20, 1, 0)
  X['small_flag'] = np.where(X['max_floor'] <= 20, 1, 0)

  # Average building height for subarea:
  sub_area_means = X.groupby('sub_area').agg({'max_floor':np.mean}).reset_index().rename(columns={'max_floor':'mean_bldg_height'})
  X = pd.merge(X, sub_area_means, on = ['sub_area'], how = 'left')

  # Sales by month:
  n_sales_months = X.groupby('year_mo').size().reset_index().rename(columns={0:'n_sales_month'})
  X = pd.merge(X, n_sales_months, on = ['year_mo'], how = 'left')

  # Average distance to Kremlin by subarea:
  dist_to_kremlin = X.groupby('sub_area').agg({'kremlin_km':np.nanmean}).reset_index().rename(columns={'kremlin_km':'subarea_dist_to_kremlin'})
  X = pd.merge(X, dist_to_kremlin, on = ['sub_area'], how = 'left')

  # Count NaNs per row:
  X['count_nan_per_row'] = X.isnull().sum(axis = 1)

  # Apartment name:
  X['apt_name'] = X['sub_area'] + X['metro_km_avto'].astype(str).apply(lambda x: x[0:5])
  X['apt_name_yrmo'] = X['apt_name'] + X['year_mo']

  #***********Missing value Imputation Start ***************************************************

  # Select all the columns having missing values less than equal to 30%
  missing_values = ((X.isna().sum())/X.shape[0])*100
  missing_values.sort_values(ascending=False,inplace = True)

  missing_values[(missing_values>0) & (missing_values<=30)].index
  values = {'prom_part_5000': X['prom_part_5000'].median(),
        'floor': X['floor'].median(),
        'railroad_station_walk_km': X['railroad_station_walk_km'].median(),
        'metro_min_walk': X['metro_min_walk'].median(),
        'metro_km_walk': X['metro_km_walk'].median(),
        'railroad_station_walk_min': X['railroad_station_walk_min'].median(),
        'product_type': X['product_type'].mode()[0],
        'green_part_2000': X['green_part_2000'].median(),
        'full_sq': X['full_sq'].median(),
        'floor_rel_total': X['floor_rel_total'].median(),
        'max_floor': X['max_floor'].median(),
        'num_room': X['num_room'].median(),
        'material': X['material'].median(),
        'preschool_quota': X['preschool_quota'].median(),
        'cafe_sum_1000_min_price_avg': X['cafe_sum_1000_min_price_avg'].median(),
        'cafe_sum_1000_max_price_avg': X['cafe_sum_1000_max_price_avg'].median(),
          'cafe_avg_price_1000': X['cafe_avg_price_1000'].median(),
        'extra_area': X['extra_area'].median(),
        'resident_to_total_ratio': X['resident_to_total_ratio'].median(),
        'extra_area_ratio': X['extra_area_ratio'].median(),
          'life_sq': X['life_sq'].median(),
          'build_count_before_1920': X['build_count_before_1920'].median(),
          'build_count_brick': X['build_count_brick'].median(),
          'build_count_1946-1970': X['build_count_1946-1970'].median(),
          'raion_build_count_with_builddate_info': X['raion_build_count_with_builddate_info'].median(),  
        'build_count_monolith': X['build_count_monolith'].median(),
        'raion_build_count_with_material_info': X['raion_build_count_with_material_info'].median(),
        'cafe_avg_price_1500': X['cafe_avg_price_1500'].median(),
        'cafe_sum_1500_min_price_avg': X['cafe_sum_1500_min_price_avg'].median(),
        'cafe_sum_1500_max_price_avg': X['cafe_sum_1500_max_price_avg'].median(),
        'cafe_avg_price_2000': X['cafe_avg_price_2000'].median(),
        'cafe_sum_2000_min_price_avg': X['cafe_sum_2000_min_price_avg'].median(),
        'cafe_sum_2000_max_price_avg': X['cafe_sum_2000_max_price_avg'].median()
        }

                      
  X.fillna(value=values,inplace=True)

  # Select all the columns having missing values between 5 to 30%
  missing_values[(missing_values>30)].index

  knn_model = pickle.load(open("model_var_knn.pkl", 'rb'))

  col = ['hospital_beds_raion','avg_room_area','state']

  for i in col:
      X[i] = knn_model[i].transform(X[[i]])


  test_id = X['id']
  X.drop(['id','timestamp'],axis=1,inplace=True)

  lis_of_all_models, reg, cat_dict = pickle.load(open("model_var.pkl", 'rb'))


  for cat in cat_dict.keys():
    X[cat] = X[cat].map(lambda s: '<unknown>' if s not in cat_dict[cat].classes_ else s)
    X[cat] = cat_dict[cat].transform(X[cat])

  test_pred = list()

  for model in lis_of_all_models:
    test_pred.append(model.predict(X.values))

  test_pred = np.array(test_pred)
  test_pred = test_pred.T

  pred = reg.predict(test_pred)
  X['price_doc'] = np.expm1(pred)
  X['id'] = test_id
  return X[['id','price_doc']]

In [5]:
def final_fun_2(X,Y):

  y_pred = final_fun_1(X)['price_doc']
  return mean_squared_log_error(y_pred, Y)

# Testing the 1st Function

In [7]:
X = pd.read_csv('test.csv')
df = final_fun_1(X)
df.head()

,id,price_doc
0,30474,5.548860e+06
1,30475,8.202937e+06
2,30476,4.882520e+06
3,30477,5.514335e+06
4,30478,5.121881e+06


# Testing the 2nd Function

In [8]:
X = pd.read_csv('train.csv')
Y = X['price_doc']
X.drop(['price_doc'],axis=1,inplace=True)

print(final_fun_2(X,Y))

0.1774656984352424
